In [39]:
# This is my work with Insaid Capstone project

There are 3 datasetts, load them one after another

Import necessary library modules

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

Load the events data set

In [85]:
df_events = pd.read_csv("events_data.csv")

In [86]:
df_events.shape

(3252950, 7)

In [87]:
df_events.info

<bound method DataFrame.info of          event_id     device_id            timestamp  longitude   latitude  \
0         2765368  2.973348e+18  2016-05-07 22:52:05  77.225676  28.730140   
1         2955066  4.734221e+18  2016-05-01 20:44:16  88.388361  22.660325   
2          605968 -3.264500e+18  2016-05-02 14:23:04  77.256809  28.757906   
3          448114  5.731369e+18  2016-05-03 13:21:16  80.343613  13.153332   
4          665740  3.388880e+17  2016-05-06 03:51:05  85.997745  23.842609   
...           ...           ...                  ...        ...        ...   
3252945   2687452 -1.937028e+18  2016-05-07 23:33:14  73.891597  18.544124   
3252946   1051580  3.345851e+18  2016-05-03 05:13:30  72.837258  19.018432   
3252947   1316227 -6.406040e+18  2016-05-01 16:03:28  77.235578  28.764065   
3252948    381262 -2.920741e+18  2016-05-05 17:22:36  83.326044  17.765488   
3252949    522592  3.212750e+18  2016-05-07 17:34:18  77.308533   9.779918   

                  city         

In [88]:
df_events.dtypes

event_id       int64
device_id    float64
timestamp     object
longitude    float64
latitude     float64
city          object
state         object
dtype: object

In [89]:
df_events.isnull().sum()

event_id       0
device_id    453
timestamp      0
longitude    423
latitude     423
city           0
state        377
dtype: int64

In [90]:
df_events.describe()

,event_id,device_id,longitude,latitude
count,3.252950e+06,3.252497e+06,3.252527e+06,3.252527e+06
mean,1.626476e+06,1.012200e+17,7.815868e+01,2.168851e+01
std,9.390459e+05,5.316758e+18,4.235570e+00,5.789111e+00
min,1.000000e+00,-9.222957e+18,1.256700e+01,8.190110e+00
25%,8.132382e+05,-4.540611e+18,7.583543e+01,1.780171e+01
50%,1.626476e+06,1.726820e+17,7.726814e+01,2.216454e+01
75%,2.439713e+06,4.861813e+18,8.031916e+01,2.868278e+01
max,3.252950e+06,9.222849e+18,9.545946e+01,4.187190e+01


In [12]:
# Connect to mysql server and get the other two datasets

In [29]:
'''
mydb = mysql.connector.connect(
  host="cpanel.insaid.co",
  user="student",
  password="student",
  database="Capstone1"
)
mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")
for x in mycursor:
  print(x)
df_gender = pd.read_sql('SELECT * FROM gender_age_train', con=mydb)
df_gender.shape
df_gender.info()
df_gender.to_csv("gender_age_train.csv", index=False)
df_phone = pd.read_sql('SELECT * FROM phone_brand_device_model', con=mydb)
df_phone.to_csv("phone_brand_device_model.csv", index=False)
'''

Consider only 6 states which are of focus

In [91]:
# States to focus for Consulting
# TamilNadu, Manipur, Chandigarh, Tripura, UttarPradesh, ArunachalPradesh
df_sixstates = df_events.loc[df_events['state'].isin(['TamilNadu','Manipur','Chandigarh','Tripura','UttarPradesh','ArunachalPradesh'])]

In [92]:
df_sixstates.shape

(534415, 7)

Check for null values

In [103]:
df_sixstates.isnull().sum()

event_id      0
device_id    63
timestamp     0
longitude    63
latitude     63
city          0
state         0
dtype: int64

There are null values, so we need to imppute them
We hall use KNN imputer to impute the null values
We shall make two data frames one for categorical vales and one for float values
Impute the float values dataframe and then combine the both dataframes

In [192]:
df1 = df_sixstates.loc[:,['event_id','timestamp','city','state']]
df2 = df_sixstates.loc[:,['device_id','longitude','latitude']]
print(df_sixstates.shape)
print(df1.shape)
print(df2.shape)
print(df1.head())

(534415, 7)
(534415, 4)
(534415, 3)
    event_id            timestamp     city      state
3     448114  2016-05-03 13:21:16  Chennai  TamilNadu
6    2948755  2016-05-03 16:20:14  Chennai  TamilNadu
10   3219828  2016-05-03 00:27:21  Chennai  TamilNadu
14    460645  2016-05-04 21:04:40  Chennai  TamilNadu
15    998037  2016-05-03 20:59:09  Chennai  TamilNadu


In [182]:
df2.isnull().sum()

device_id    63
longitude    63
latitude     63
dtype: int64

In [183]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
df2_imputed = imputer.fit_transform(df2)

In [184]:
print (df2_imputed.shape)
df2_imp = pd.DataFrame(df2_imputed, columns=['device_id','longitude','latitude'])
print(df2_imp.shape)
print (df2_imp.isnull().sum())
print(df2_imp.head())

(534415, 3)
(534415, 3)
device_id    0
longitude    0
latitude     0
dtype: int64
      device_id  longitude   latitude
0  5.731369e+18  80.343613  13.153332
1  5.536513e+18  80.359796  13.128174
2 -3.433061e+18  80.290779  13.115427
3  7.983871e+18  80.335435  13.189053
4  7.594216e+18  80.306871  13.172970


Merge the two dataframes back to main dataframe

In [196]:
df1.reset_index(drop=True, inplace=True)
print(df1.head())
print(df2_imp.head())

   event_id            timestamp     city      state
0    448114  2016-05-03 13:21:16  Chennai  TamilNadu
1   2948755  2016-05-03 16:20:14  Chennai  TamilNadu
2   3219828  2016-05-03 00:27:21  Chennai  TamilNadu
3    460645  2016-05-04 21:04:40  Chennai  TamilNadu
4    998037  2016-05-03 20:59:09  Chennai  TamilNadu
      device_id  longitude   latitude
0  5.731369e+18  80.343613  13.153332
1  5.536513e+18  80.359796  13.128174
2 -3.433061e+18  80.290779  13.115427
3  7.983871e+18  80.335435  13.189053
4  7.594216e+18  80.306871  13.172970


In [197]:
df_sixstates_imputed = pd.concat([df1,df2_imp], axis=1)
print (df_sixstates_imputed.shape)
print (df_sixstates_imputed.isnull().sum())
df_sixstates_imputed.head()

(534415, 7)
event_id     0
timestamp    0
city         0
state        0
device_id    0
longitude    0
latitude     0
dtype: int64


,event_id,timestamp,city,state,device_id,longitude,latitude
0,448114,2016-05-03 13:21:16,Chennai,TamilNadu,5.731369e+18,80.343613,13.153332
1,2948755,2016-05-03 16:20:14,Chennai,TamilNadu,5.536513e+18,80.359796,13.128174
2,3219828,2016-05-03 00:27:21,Chennai,TamilNadu,-3.433061e+18,80.290779,13.115427
3,460645,2016-05-04 21:04:40,Chennai,TamilNadu,7.983871e+18,80.335435,13.189053
4,998037,2016-05-03 20:59:09,Chennai,TamilNadu,7.594216e+18,80.306871,13.172970


In [198]:
import folium

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=5,
                        weight=0,
                        fill_color='#000000').add_to(this_map)


In [199]:
df_tmp = df_sixstates_imputed.loc[df_sixstates_imputed['state'].isin(['TamilNadu'])]

In [200]:
df_tmp.isnull().sum()

event_id     0
timestamp    0
city         0
state        0
device_id    0
longitude    0
latitude     0
dtype: int64

In [201]:
df_tmp.shape

(436282, 7)

In [202]:
# Get unique values of lat,long to see if we can reduce no of rows and also to map the same
df_tmp1 = df_tmp.groupby(['latitude','longitude']).size().reset_index(name='Freq')

In [203]:
df_tmp1.shape

(5773, 3)

In [204]:
#create a map
this_map = folium.Map(prefer_canvas=True)

#use df.apply(,axis=1) to iterate through every row in your dataframe
df_tmp1.apply(plotDot, axis = 1)
#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())
this_map